In [1]:
# データ分析・操作用ライブラリ
import numpy as np
import pandas as pd
from collections import Counter
import itertools

# NLP用ライブラリ
import MeCab,re
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
# CSVの読み込み
wikiData = pd.read_csv("csv/train.csv", encoding="utf-8")

# 読み込みデータの表示
wikiData.head()

,text,target
0,ドール・フード・カンパニー ドール・フード・カンパニー（Dole Food Compan...,1
1,南ヶ丘牧場 株式会社 南ヶ丘牧場（みなみがおかぼくじょう）は、栃木県那須高原に本拠を置く...,1
2,岩瀬牧場 岩瀬牧場（いわせぼくじょう）は、日本の福島県岩瀬郡鏡石町にある牧場である。 ...,1
3,"クラギ クラギ株式会社（くらぎ、""KURAGI Co.Ltd."" ）は、三重県を中心に「...",1
4,"ハッピーネモファーム 株式会社ハッピーネモファーム（""""）は、北海道浦河郡浦河町にある競...",1


In [3]:
# 形態素解析用関数
def tokenize(text,wordClassList):
    tokens = []
    tagger = MeCab.Tagger()
    node = tagger.parseToNode(text)
    while node:
        if node.feature.split(",")[0] in wordClassList :
                replace_node = re.sub( re.compile( "[!-/:-@[-`{-~]" ), "", node.surface )
                if replace_node != "" and replace_node != " ":
                    tokens.append(replace_node)
        node = node.next
    return tokens

In [4]:
cleanedData = wikiData.copy()

# 数字の除外
cleanedData['text'] = cleanedData['text'].str.replace('[0-9]+','', regex=True)

# 不要な記号の除外
path = "data/Removal Symbols.txt"
removal_symbols = []
with open(path, encoding="utf-8") as f:
    removal_symbols = [s.strip() for s in f.readlines()]
print(removal_symbols)

for symbol in removal_symbols:
    cleanedData['text'] = cleanedData['text'].str.replace(symbol,'')

['(', ')', '（', '）', '"', '""', '「', '」', '[', ']', '', '', '']


In [5]:
cleanedData.head()

,text,target
0,ドール・フード・カンパニー ドール・フード・カンパニーDole Food Company...,1
1,南ヶ丘牧場 株式会社 南ヶ丘牧場みなみがおかぼくじょうは、栃木県那須高原に本拠を置く酪農...,1
2,岩瀬牧場 岩瀬牧場いわせぼくじょうは、日本の福島県岩瀬郡鏡石町にある牧場である。 鏡石...,1
3,クラギ クラギ株式会社くらぎ、KURAGI Co.Ltd. は、三重県を中心に農業屋など...,1
4,ハッピーネモファーム 株式会社ハッピーネモファームは、北海道浦河郡浦河町にある競走馬サラ...,1


In [6]:
# Numpy配列へ変換
X = cleanedData.text.values
y = cleanedData.target.values

In [7]:
# 形態素解析
wordClassList = ['名詞']
words = []
for text in X:
    words.append(tokenize(text,wordClassList))
    
print(len(words))

11460


In [8]:
# 1次元配列へ変換
words_1d = itertools.chain.from_iterable(words)

In [9]:
# 単語別の登場回数をカウント
delWord = []
counter = Counter(words_1d)
for word, count in counter.most_common():
    if count == 1:
        delWord.append(word)
    #print(f"{word}: {count}")

In [12]:
# 登場回数の低い単語を削除
for dw in delWord:
    for i in range(len(words)): 
        words[i] = [word for word in words[i] if word != dw]

In [13]:
# ストップワードの読み込み
path = "data/Slothlib.txt"
stopWords = []
with open(path, encoding="utf-8") as f:
    stopWords = [s.strip() for s in f.readlines()]

# ストップワードの削除
for sw in stopWords:
    for i in range(len(words)): 
        words[i] = [word for word in words[i] if word != sw]

In [14]:
# 分かち書きを行ったテキストへ変換
cleanedData['text'] = list(map(lambda text:str(text) , words))
cleanedData['text'] = cleanedData['text'].str.replace(","," ")
cleanedData['text'] = cleanedData['text'].str.replace("'","")
cleanedData['text'] = cleanedData['text'].str.replace("[","")
cleanedData['text'] = cleanedData['text'].str.replace("]","")

In [22]:
# 欠損値の削除
cleanedData['text'] = cleanedData['text'].replace("", np.nan)
cleanedData.dropna(inplace=True)

In [23]:
# CSV出力
cleanedData.to_csv("csv/train_cleaned.csv",index = False, encoding='utf-8')